# Key Term Search

This notebook contains Python code for searching for key terms of the three audit teams: Performance; Local Government; and ICT.

## Import Data

In [9]:
import pandas as pd
import numpy as np
import time

In [10]:
# Import data from spreadsheet
data = pd.read_excel ("..\\data\\Hansard22102019.xlsx", sheet_name="Text")
text = pd.DataFrame(data, columns= ['HansardID','TextID','Text'])
text = text.astype({"HansardID":'str', "TextID":'str', "Text":'str'}) 
text['TextLower'] = text.Text.str.lower() # Convert to lowercase

text.head(2)

,HansardID,TextID,Text,TextLower
0,HANSARD-10-26147.xml,201903209d1c421954fd410090000001,Climate Change,climate change
1,HANSARD-10-26147.xml,201903209d1c421954fd410090000002,The Hon. M.C. PARNELL (14:55): I seek leave t...,the hon. m.c. parnell (14:55): i seek leave t...


In [22]:
def process_term(term):
    if "?=.*" in term:      
        # Search for terms separated by any text and in any order
        term = r'(?=.*\b' + term + r'\b)'
    elif not term:
        # Term does not exist and should not be processed further
        return "" 
    else:
        # Search for term as a whole word    
        term = r'\b(' + term + ')' + r'\b' # \b word boundary
    
    return term

def process_terms(terms):

    # https://docs.microsoft.com/en-us/dotnet/standard/base-types/quantifiers-in-regular-expressions
    # http://www.rexegg.com/regex-quickstart.html
        
    processed = terms.str.lower() # Convert to lowercase
    processed = processed.str.strip() # Trim any whitespace from ends
        
    # Search for terms separated by '+' in any order: Use regex (?=.*\bTerm\b)(?=.*\bTerm\b) where \b is a word boundary
    processed = processed.str.replace(' *\+ *', r'\\b)(?=.*\\b')
    processed = processed.apply(lambda x: process_term(x))
                
    return processed

def import_spreadsheet_terms(path, sheet_name):
    excel_data = pd.read_excel(path, sheet_name=sheet_name)
    excel_data = pd.DataFrame(excel_data)
    excel_data['Alternate'] = excel_data.Alternate.replace(np.nan, '', regex=True)  # Replace missing alternate terms
    excel_data = excel_data.astype({"Term": 'str', "Alternate": 'str'})
    excel_data['TermPattern'] = process_terms(excel_data.Term)  # Process term into regular expression (regex)
    excel_data['AlternatePattern'] = process_terms(excel_data.Alternate)  # Process alternate term into regex
    return excel_data

In [23]:
performance = import_spreadsheet_terms("..\\data\\AuditTeamTerms.xlsx", "Performance")

print(performance.TermPattern.head(25), "\n")
print(performance.AlternatePattern.head(5))

0                   \b(misappropriation)\b
1                  \b(maladministration)\b
2                         \b(misconduct)\b
3                              \b(error)\b
4                              \b(fraud)\b
5                   \b(royal commission)\b
6                            \b(inquiry)\b
7                        \b(outsourcing)\b
8                      \b(privatisation)\b
9                            \b(probity)\b
10                     \b(national park)\b
11                      \b(speed camera)\b
12                     \b(safety camera)\b
13                       \b(kordamentha)\b
14                  \b(elective surgery)\b
15                 \b(operating theatre)\b
16                         \b(home care)\b
17                         \b(gp centre)\b
18           \b(building better schools)\b
19      \b(sustainable enrolment growth)\b
20    (?=.*\beducation\b)(?=.*\bcapital\b)
21    (?=.*\byear 7\b)(?=.*\btransition\b)
22                     \b(adelaide oval)\b
23         

In [24]:
government = import_spreadsheet_terms("..\\data\\AuditTeamTerms.xlsx", "Local Government")

print(government.TermPattern.head(5), "\n")
print(government.AlternatePattern.head(5))

0                         \b(council)\b
1    \b(local government association)\b
2      \b(office of local government)\b
3                \b(local government)\b
4                 \b(planning reform)\b
Name: TermPattern, dtype: object 

0    \b(councils)\b
1         \b(lga)\b
2         \b(olg)\b
3                  
4                  
Name: AlternatePattern, dtype: object


In [25]:
it = import_spreadsheet_terms("..\\data\\AuditTeamTerms.xlsx", "IT")

print(it.TermPattern.head(5), "\n")
print(it.AlternatePattern.head(5))

0            \b(ict)\b
1     \b(it project)\b
2    \b(ict project)\b
3     \b(it program)\b
4    \b(ict program)\b
Name: TermPattern, dtype: object 

0           \b(information communication technology)\b
1                 \b(information technology project)\b
2    \b(information communication technology projec...
3                 \b(information technology program)\b
4    \b(information communication technology progra...
Name: AlternatePattern, dtype: object


## Search for Key Terms in Hansard Text

In [26]:
def search_key_term(terms_df, text_df, audit_team_name):
    
    results = pd.DataFrame().reindex_like(text_df)
    results['Term'] = np.NaN
    del results['Text'] # Delete unneeded Text column from results
    del results['TextLower'] # Delete unneeded Text column from results
    results = results.dropna() # Delete all rows

    for index, row in terms_df.iterrows():
        # Search for match to term
        match_term = text_df.TextLower.str.contains(row['TermPattern'], case=False, regex=True) | \
                     text_df.Text.str.contains(row['TermPattern'], case=False, regex=True)       
        
        match = pd.DataFrame(match_term, columns=['Term'])   
        
        # Search for match to alternate term
        if row['AlternatePattern']:
            # Match to alternate term is true if a match has already been found for the original term
            match_alternate = match['Term'] | \
                              text_df.TextLower.str.contains(row['AlternatePattern'], case=False, regex=True) | \
                              text_df.Text.str.contains(row['AlternatePattern'], case=False, regex=True)
            match_alternate = pd.DataFrame(match_alternate, columns=['Alternate'])
            match['Term'] = match['Term'] | match_alternate['Alternate']
        
        if not match.empty:
            match = pd.concat([text_df.reset_index(drop=True), match], axis=1)
            del match['Text'] # Delete unneeded Text column from results
            del match['TextLower'] # Delete unneeded TextLower column from results
            match = match.loc[match.Term, :] # drop rows that did not match term            
            match['Term'] = row['Term']
            results = pd.concat([results, match], ignore_index=True, sort=False) # Add matched terms to final result
    
    results = results.drop_duplicates() # Drop duplicate rows
    results['AuditTeam'] = audit_team_name
    return results

In [27]:
start_time = time.time()

performance_terms = search_key_term(performance, text, "Performance") # Performance Audit Team Terms
government_terms = search_key_term(government, text, "Local Government") # Local Government Audit Team Terms
it_terms = search_key_term(it, text, "IT") # IT Audit Team Terms

end_time = time.time()
total_time = end_time-start_time
print(total_time)

3866.8759281635284


In [28]:
# Merge results and output to Excel
merged_data = pd.concat([performance_terms, government_terms,it_terms], ignore_index=True)
merged_data = merged_data.drop_duplicates() # Drop duplicate rows

# Without alternate term search: 17873 results. With alternate term search: 19040 3883.3312656879425 sec
print("Combined = ", merged_data.shape)
merged_data.to_excel('.\\TermSearch.xlsx', sheet_name='TermSearch', index=False)

Combined =  (19040, 4)
